In [1]:
import pwd
from IPython import get_ipython

get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

/tmp/ipykernel_2811/2117665699.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
/tmp/ipykernel_2811/2117665699.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')


In [2]:
from typing import Sequence

from flax import linen as nn
from jax import random, value_and_grad
import jax.numpy as jnp
from scipy.io import arff

from src.dbopt.FCNN import FCNN

In [3]:
seed = 24
key = random.PRNGKey(seed)

from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


### Importing the data

In [4]:
data = arff.loadarff('data/column_2C_weka.arff')[0]

def row_void_to_array(entry):
    return jnp.array([entry[i] for i in range(6)])

key, ds_key = random.split(key)

x = jnp.array(list(map(row_void_to_array, data)))
y = jnp.array(list(map(lambda row : 0 if row[6]==b'Normal' else 1, data)))
dataset = random.permutation(ds_key, jnp.concatenate((jnp.expand_dims(y, axis=1), x), axis=1), axis=0)

print(dataset.shape)

(310, 7)


### Fitting a the network

In [5]:
model = FCNN(num_neurons_per_layer=[100, 100, 100, 2])
key, init_x_key = random.split(key)
x_init = random.uniform(init_x_key, (6,))
key, init_key = random.split(key)
params = model.init(init_key, x_init)

key, train_key = random.split(key)
params = model.train(train_key, params, dataset, 300, lr=0.0001, logs_frequency=10)

epoch 0, loss = 6.078137397766113, training accuracy = [0.5032258]
epoch 10, loss = 0.8592239022254944, training accuracy = [0.6935484]
epoch 20, loss = 0.34373438358306885, training accuracy = [0.82580644]
epoch 30, loss = 0.37720781564712524, training accuracy = [0.86774194]
epoch 40, loss = 0.5584328174591064, training accuracy = [0.83548385]
epoch 50, loss = 0.32912495732307434, training accuracy = [0.85806453]
epoch 60, loss = 0.3469810485839844, training accuracy = [0.85806453]
epoch 70, loss = 0.3820207118988037, training accuracy = [0.86129034]
epoch 80, loss = 0.33947569131851196, training accuracy = [0.81935483]
epoch 90, loss = 0.2754552960395813, training accuracy = [0.86774194]
epoch 100, loss = 0.24525286257266998, training accuracy = [0.85161287]
epoch 110, loss = 0.3184654116630554, training accuracy = [0.86129034]
epoch 120, loss = 0.3614353537559509, training accuracy = [0.85483867]
epoch 130, loss = 0.28241774439811707, training accuracy = [0.86451614]
epoch 140, los